In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.layers.convolutional import Convolution2D, AveragePooling2D
from keras import callbacks
import numpy as np

Using TensorFlow backend.


In [0]:

"""
Parameters
"""
epochs = 100
img_width, img_height =300,300
classes_num = 5
batch_size=32
arsitektur = 'LeNet5'
#lr = 0.0004

In [0]:
DATA_DIR = '/content/gdrive/My Drive/CITS Color'

train_datagen = ImageDataGenerator(rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    DATA_DIR, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data


Found 1079 images belonging to 5 classes.
Found 268 images belonging to 5 classes.


In [0]:


# (3) Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Convolution2D(filters=6, input_shape=(img_width,img_height,3), kernel_size=(3,3)))
model.add(Activation('relu'))
# Pooling
model.add(AveragePooling2D())
# Batch Normalisation
model.add(BatchNormalization())
          

# 2nd Convolutional Layer
model.add(Convolution2D(filters=16, kernel_size=(3,3)))
model.add(Activation('relu'))
# Pooling
model.add(AveragePooling2D())
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(120))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(84))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(classes_num))
model.add(Activation('softmax'))

model.summary()


architecure_dir = os.path.join('/content/gdrive/My Drive/Colab Notebooks/LeafClassification/',arsitektur)
if not os.path.exists(architecure_dir):
  os.mkdir(architecure_dir)
# Save the model architecture
with open(os.path.join(architecure_dir,'architecture.json'), 'w') as f:
    f.write(model.to_json())



Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 298, 6)       168       
_________________________________________________________________
activation_1 (Activation)    (None, 298, 298, 6)       0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 149, 149, 6)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 149, 149, 6)       24        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 147, 147, 16)      880       
_________________________________________________________________
activation_2 (Activation)    (None, 147, 147, 16)      0         
_________________________________________________________________
aver

In [0]:
from keras.models import model_from_json
with open(os.path.join(architecure_dir,'architecture.json'), 'r') as f:
    model = model_from_json(f.read())

In [0]:
# (4) Compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
"""
Tensorboard log
"""


log_dir = os.path.join(architecure_dir, 'tf-log')
filechekcpoint = "saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True)
#checkpoint = callbacks.ModelCheckpoint(filepath=os.path.join(log_dir, filechekcpoint), monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
#cbks = [tb_cb,checkpoint]
cbks = [tb_cb]

"""model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=cbks,
    validation_steps=validation_steps)"""

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    callbacks=cbks,
    epochs = epochs,
    shuffle = True)

#model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), steps_per_epoch=len(x_train) / 32, epochs=epochs)

target_dir =os.path.join(architecure_dir, 's')

if not os.path.exists(target_dir):
  print(target_dir)
  os.mkdir(target_dir)
  
model.save(os.path.join(target_dir, 'model.h5'))
model.save_weights(os.path.join(target_dir, 'weights.h5'))

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
33/33 [==============================] - 578s 18s/step - loss: 0.9180 - acc: 0.6629 - val_loss: 0.9952 - val_acc: 0.6523
Epoch 2/100
33/33 [==============================] - 37s 1s/step - loss: 0.6030 - acc: 0.7768 - val_loss: 12.0203 - val_acc: 0.2542
Epoch 3/100
33/33 [==============================] - 27s 811ms/step - loss: 0.4916 - acc: 0.8142 - val_loss: 12.4984 - val_acc: 0.2246
Epoch 4/100
33/33 [==============================] - 27s 812ms/step - loss: 0.4227 - acc: 0.8449 - val_loss: 12.4984 - val_acc: 0.2246
Epoch 5/100
33/33 [==============================] - 27s 807ms/step - loss: 0.3756 - acc: 0.8563 - val_loss: 9.4671 - val_acc: 0.3941
Epoch 6/100
33/33 [==============================] - 27s 806ms/step - loss: 0.3415 - acc: 0.8822 - val_loss: 12.3618 - val_acc: 0.2331
Epoch 7/100
33/33 [==============================] - 27s 819ms/step - loss: 0.2833 - acc: 0.8862 - val_loss: 12.4984 - val_acc: 0.2246
Epoch 8/100
3